# GSoC pretest

## Imports

In [ ]:
import torch
import timm
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import torch.quantization as quantization
import matplotlib.pyplot as plt
from openvino.runtime import Core
from openvino.runtime import serialize
from openvino.tools import mo
import onnx
import onnxruntime as ort
from onnxruntime.quantization import QuantType, quantize_dynamic
from openvino.tools.pot import IEEngine, load_model, save_model, compress_model_weights, create_pipeline
from openvino.tools.pot.algorithms.quantization.default.algorithm import DefaultQuantization
from openvino.tools.pot.api import DataLoader
from openvino.runtime import Core
import os
import cv2 as cv
from pathlib import Path

## Load Model

In [ ]:
model_names = timm.list_models('swin_small*')
for model_name in model_names:
    print(model_name)
model = timm.create_model('swin_small_patch4_window7_224', pretrained=True)

## Run model at ont-quantization

In [ ]:
# Load image and preprocess
image = Image.open('../notebooks/data/image/coco.jpg')
# plt.imshow(image)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.299, 0.224, 0.255])
])
image = transform(image).unsqueeze(0)

# Predict the class of image
with torch.no_grad():
    output = model(image)
    pred = output.argmax(dim=1).item()
    print(f'Predicted class: {pred}')
# After searching, the 208th category in ImageNet is dog.

## Define convert model to onnx and IR function

In [ ]:
def convert_models(model, model_input, path):
    script_model = torch.jit.trace(model, model_input)
    torch.onnx.export(script_model,  model_input, path)   
    convert_model = mo.convert_model(path)
    # Change the .onnx suffix to .xml
    IR_path = path[:-4]+'xml'
    serialize(convert_model, IR_path)

## Convert model

In [ ]:
# Convert model to onnx and IR
# If you already have a models directory, place comment the next line of code.
!mkdir models
model.eval()
onnx_path = 'models/swin_small_patch4_window7_224.onnx'
model_input = torch.randn(1,3,224,224).cpu() 
convert_models(model, model_input,onnx_path)

### quantize pytorch model

In [ ]:
model_int8 = quantization.quantize_dynamic(model, dtype=torch.qint8)
with torch.no_grad():
    output = model_int8(image)
    pred = output.argmax(dim=1).item()
    print(f'Predicted class: {pred}')
torch.save(model_int8, 'models/quantize_swin.pth')

### quantize onnx model

In [ ]:
quantize_path = 'models/quantize_swin.onnx'
quantize_dynamic(model_input=onnx_path,
    model_output= quantize_path,
    weight_type=QuantType.QInt8,
    optimize_model=True
)

In [ ]:
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model, full_check=True)

### quantize openvino model

In [ ]:
class SwinDataLoader(DataLoader):
    def __init__(self, dataset_path):
        self._files = []
        all_files_in_dir = os.listdir(dataset_path)
        for name in  all_files_in_dir:
            file = os.path.join(dataset_path, name)
            if cv.haveImageReader(file):
                self._files.append(file)

        self._shape = (1,3, 224,224)

    def __len__(self):
        return len(self._files)

    def __getitem__(self, index):
        image =  Image.open(self._files[index])
        image = transform(image).unsqueeze(0)
        return image, None


In [ ]:
algorithm_config = [{
    'name': 'DefaultQuantization',
    'params':{
        'target_device': 'ANY',
        'stat_subset_size':300,
        'stat_batch_size':1,
    },
}]
model_config = {
    "model_name": "model",
    "model": 'models/swin_small_patch4_window7_224.xml',
    "weights": 'models/swin_small_patch4_window7_224.bin'
}
ie = Core()
engine_config = {'device': 'CPU'}

data_loader = SwinDataLoader('images/')

openvino_model = load_model(model_config=model_config)

engine = IEEngine(config=engine_config, data_loader=data_loader)
pipline = create_pipeline(algorithm_config, engine)
compressed_model = pipline.run(model=openvino_model)
compress_model_weights(compressed_model)
compressed_model_paths = save_model(
    model=compressed_model,
    save_path='models/',
    model_name = 'quantization_swin'
)

### Compara the Size of the Original and Quantized Models

In [ ]:
original_model_size = Path('models/swin_small_patch4_window7_224.bin').stat().st_size / (1024 * 1024)
quantized_model_size = Path('models/quantization_swin.bin').stat().st_size / (1024 * 1024)

print(f'Original IR  size: {original_model_size:.2f} MB')
print(f'Quantized IR size: {quantized_model_size:.2f} MB')

In [ ]:
original_model_size = Path('./models/swin_small_patch4_window7_224.onnx').stat().st_size / (1024 * 1024)
quantized_model_size = Path('./models/quantize_swin.onnx').stat().st_size / (1024 * 1024)

print(f'Original ONNX Model size: {original_model_size:.2f} MB')
print(f'Quantized ONNX Model size: {quantized_model_size:.2f} MB')

### Compare Performance of the Original and Quantized Models

In [ ]:
!benchmark_app --help

In [ ]:
!benchmark_app -m models/swin_small_patch4_window7_224.xml -d CPU -t 15 -api sync

In [ ]:
!benchmark_app -m models/quantization_swin.xml -d CPU -t 15 -api sync

## quantize to 4 bit

In [68]:
algorithm_config = [{
    'name': 'DefaultQuantization',
    'params':{
        'target_device': 'ANY',
        'stat_subset_size':300,
        'stat_batch_size':1,
        'bits':4,
    },
}]
openvino_model = load_model(model_config=model_config)
pipline = create_pipeline(algorithm_config, engine)
compressed_model = pipline.run(model=openvino_model)
compress_model_weights(compressed_model)
compressed_model_paths = save_model(
    model=compressed_model,
    save_path='models/',
    model_name = 'quantization4_swin'
)

/home/user/miniconda3/envs/openvino_env/lib/python3.8/site-packages/openvino/tools/pot/algorithms/quantization/utils.py:318: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  input_shape = mode(activations_statistics[input_node_name]['shape'])[0][0]


In [ ]:
original_model_size = Path('models/quantization4_swin.bin').stat().st_size / (1024 * 1024)
quantized_model_size = Path('models/quantization_swin.bin').stat().st_size / (1024 * 1024)

print(f'4int IR  size: {original_model_size:.2f} MB')
print(f'8int IR size: {quantized_model_size:.2f} MB')